# START YOUR CODE HERE IN THIS NOTEBOOK, SIR/MS. :)

In [123]:
import os

modelList = [] 

for x in os.listdir("model"): 
    modelList += [x]
    
print(modelList)

['archive', 'PDAN_TSU_RGB.zip']


In [124]:
import zipfile, torch
import ipywidgets as widgets
from ipywidgets import interact, Dropdown 
from IPython.display import display

models = widgets.Dropdown(options = modelList)
load_button = widgets.Button(description="Load")
output = widgets.Output()
selected_model = ""

In [125]:
def onClickLoadModel(b):
    selected_model = models.value
    if (selected_model.split('.')[-1] == "zip"):
        with zipfile.ZipFile("model/"+selected_model,"r") as zip_file:
            zip_file.extractall("model/")
    for x in os.listdir("model/archive"):
        if (x == "data.pkl"):
            pretrained_model = torch.load("data.pkl")

In [126]:
load_button.on_click(onClickLoadModel)

In [127]:
widgets.HBox([models,load_button])